In [6]:
from gurobipy import *
import numpy as np
import random as rnd

In [1]:
def initializeData():
    rnd.seed( 1 )
    noFacs = 16
    noCusts = 32
    facs = np.zeros( ( noFacs , 2 ) )
    for i in range( noFacs ):
        facs[ i ][ 0 ] = rnd.uniform( 0 , 100 )
        facs[ i ][ 1 ] = rnd.uniform( 0 , 100 )
        custs = np.zeros( ( noCusts , 2 ) )
    for j in range( noCusts ):
        custs[ j ][ 0 ] = rnd.uniform( 0 , 100 )
        custs[ j ][ 1 ] = rnd.uniform( 0 , 100 )
        fixs = np.zeros( noFacs )
    for i in range( noFacs ):
        fixs[ i ] = rnd.uniform( 4000 , 12000 )
        caps = np.zeros( noFacs )
    for i in range( noFacs ):
        caps[ i ] = rnd.uniform( 0 , 800 )
        dems = np.zeros( noCusts )
    for j in range( noCusts ):
        dems[ j ] = rnd.uniform( 0 , 100 )
    return( noFacs , noCusts , facs , custs , fixs , caps , dems )

In [2]:
noFacs , noCusts , facs , custs , fixs , caps , dems = initializeData()
facs

array([[13.43642441, 84.74337369],
       [76.3774619 , 25.50690257],
       [49.54350871, 44.94910648],
       [65.15929727, 78.87233511],
       [ 9.38595868,  2.83474765],
       [83.57651039, 43.27670679],
       [76.22800825,  0.21060534],
       [44.53871941, 72.15400323],
       [22.87622213, 94.52706956],
       [90.14274576,  3.0589983 ],
       [ 2.5445861 , 54.14124728],
       [93.91491628, 38.12042377],
       [21.65993971, 42.21165756],
       [ 2.90407876, 22.16916663],
       [43.78875937, 49.58122414],
       [23.30844503, 23.08665415]])

In [3]:
def distance( i , j ):
    global facs , custs
    dx = facs[ i ][ 0 ] - custs[ j ][ 0 ]
    dy = facs[ i ][ 1 ] - custs[ j ][ 1 ]
    val = ( ( dx * dx ) + ( dy * dy ) ) ** 0.5
    return val

In [8]:
def constructVars():
    global noFacs , noCusts , dems , myModel , yVars , xVars
    for i in range( noFacs ):
        curVar = myModel.addVar( vtype = GRB.BINARY , ub = 1 , name = "y" + str( i ) )
        yVars[ i ] = curVar
    for i in range( noFacs ):
        for j in range( noCusts ):
            curVar = myModel.addVar( vtype = GRB.CONTINUOUS , ub = dems[ j ] , name = "x"
    + str( i ) + "_" + str( j ) )
            xVars[ i ][ j ] = curVar
    myModel.update()
    
    
def constructObj():
    global noFacs, noCusts, fixs , myModel, yVars, xVars
    objExpr = LinExpr()
    for i in range( noFacs ):
        objExpr += fixs[ i ] * yVars[ i ]
    for i in range( noFacs ):
        for j in range( noCusts ):
            objExpr += distance( i , j ) * xVars[ i ][ j ]
    myModel.setObjective( objExpr , GRB.MINIMIZE )

def constructConstrs():
    global noFacs, noCusts, facs, caps , dems, myModel , yVars, xVars
    for j in range( noCusts ):
        constrExpr = LinExpr()
        for i in range( noFacs ):
            constrExpr += 1 * xVars[ i ][ j ]
        myModel.addConstr( lhs = constrExpr , sense = GRB.EQUAL , rhs = dems[ j ] , name
    = "d" + str( j ) )
    for i in range( noFacs ):
        constrExpr = LinExpr()
        for j in range( noCusts ):
            constrExpr += 1 * xVars[ i ][ j ]
        constrExpr += ( - caps[ i ] * yVars[ i ] )
        myModel.addConstr( lhs = constrExpr, sense = GRB.LESS_EQUAL , rhs = 0 , name= "c"
    + str( i ) )
    myModel.update()

In [9]:
( noFacs , noCusts , facs , custs , fixs , caps , dems ) = initializeData()
myModel = Model( "facility_location" )
yVars = [ 0 for i in range( noFacs ) ]
xVars = [ [ 0 for j in range ( noCusts ) ] for i in range ( noFacs ) ]
constructVars()
constructObj()
constructConstrs()
myModel.write( "facility_loc_lp.lp" )
myModel.optimize()
print ( "Optimal objective: " + str( myModel.ObjVal ) )

Optimize a model with 48 rows, 528 columns and 1040 nonzeros
Variable types: 512 continuous, 16 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  Objective range  [2e+00, 1e+04]
  Bounds range     [1e+00, 1e+02]
  RHS range        [2e+00, 1e+02]
Found heuristic solution: objective 169684.58347
Presolve time: 0.00s
Presolved: 48 rows, 528 columns, 1040 nonzeros
Variable types: 512 continuous, 16 integer (16 binary)

Root relaxation: objective 3.820724e+04, 27 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 38207.2366    0    8 169684.583 38207.2366  77.5%     -    0s
H    0     0                    82021.878932 38207.2366  53.4%     -    0s
H    0     0                    74841.424921 38207.2366  48.9%     -    0s
     0     0 43819.3134    0    9 74841.4249 43819.3134  41.5%     -    0s
     0     0 45629.2763    0  

In [63]:
def constructVarsPairwise():
    global noFacs , noCusts , dems , myModel , yVars , xVars
#     for i in range( noFacs ):
#         curVar = myModel.addVar( vtype = GRB.BINARY , ub = 1 , name = "y" + str( i ) )
#         yVars[ i ] = curVar
    for i in range( noFacs ):
        for j in range( noCusts ):
            curVar = myModel.addVar( vtype = GRB.CONTINUOUS , ub = dems[ j ] , name = "x"
    + str( i ) + "_" + str( j ) )
            xVars[ i ][ j ] = curVar
    myModel.update()

In [80]:
def calculateCost(S_try):
    global noFacs , noCusts , dems , myModel , yVars , xVars
    print(S_try)
    myModel = Model( "Pairwise" )
    myModel.Params.OutputFlag = 0
    yVars = [ 0 for i in range( noFacs ) ]
    for i in S_try:
        yVars[i] = 1
    xVars = [ [ 0 for j in range ( noCusts ) ] for i in range ( noFacs ) ]
    constructVarsPairwise()
    constructObj()
    constructConstrs()
#     myModel.write( "facility_loc_lp.lp" )
    myModel.optimize()
    if myModel.status == 3:
        return 10**6 
    else:
        return myModel.ObjVal
    
    

In [118]:
### pairwise exchange implementation ###

def pairwise(M, rand):
    global noFacs , noCusts , dems , myModel , yVars , xVars
    rnd.seed()
    if rand:
        S_current = set(rnd.sample(M,10))
    else:
        S_current = M
    S_try = {}
    improvement = True
    while improvement:
        best_sol = {}
        best_cost = 10**6
        print(S_current)
        for facility in M:
            if facility in S_current:
                S_try = S_current - {facility}
            else:
                S_try = S_current | {facility}
            current_cost = calculateCost(S_try)
            if current_cost <= best_cost:
                best_sol = S_try
                best_cost = current_cost
#         print(best_sol , S_current)
        if calculateCost(best_sol) <= calculateCost(S_current):
            S_current = best_sol
            improvement = True
            print('best cost ', calculateCost(S_current))
        else:
            improvement = False
    return S_current

In [120]:
( noFacs , noCusts , facs , custs , fixs , caps , dems ) = initializeData()
yVars = [ 0 for i in range( noFacs ) ]
xVars = [ [ 0 for j in range ( noCusts ) ] for i in range ( noFacs ) ]
M = set(range(16))
optimal_facilities = pairwise(M, rand = True)
print(optimal_facilities)
myModel = Model( "pairwise" )
yVars = [ 0 for i in range( noFacs ) ]
for i in optimal_facilities:
    yVars[i] = 1
xVars = [ [ 0 for j in range ( noCusts ) ] for i in range ( noFacs ) ]
constructVarsPairwise()
constructObj()
constructConstrs()
myModel.write( "facility_pairwise.lp" )
myModel.optimize()
print ( "Optimal objective: " + str( myModel.ObjVal ) )

{0, 2, 3, 4, 6, 7, 9, 10, 12, 15}
{2, 3, 4, 6, 7, 9, 10, 12, 15}
{0, 1, 2, 3, 4, 6, 7, 9, 10, 12, 15}
{0, 3, 4, 6, 7, 9, 10, 12, 15}
{0, 2, 4, 6, 7, 9, 10, 12, 15}
{0, 2, 3, 6, 7, 9, 10, 12, 15}
{0, 2, 3, 4, 5, 6, 7, 9, 10, 12, 15}
{0, 2, 3, 4, 7, 9, 10, 12, 15}
{0, 2, 3, 4, 6, 9, 10, 12, 15}
{0, 2, 3, 4, 6, 7, 8, 9, 10, 12, 15}
{0, 2, 3, 4, 6, 7, 10, 12, 15}
{0, 2, 3, 4, 6, 7, 9, 12, 15}
{0, 2, 3, 4, 6, 7, 9, 10, 11, 12, 15}
{0, 2, 3, 4, 6, 7, 9, 10, 15}
{0, 2, 3, 4, 6, 7, 9, 10, 12, 13, 15}
{0, 2, 3, 4, 6, 7, 9, 10, 12, 14, 15}
{0, 2, 3, 4, 6, 7, 9, 10, 12}
{2, 3, 4, 6, 7, 9, 10, 12, 15}
{0, 2, 3, 4, 6, 7, 9, 10, 12, 15}
{2, 3, 4, 6, 7, 9, 10, 12, 15}
best cost  89883.42192759288
{2, 3, 4, 6, 7, 9, 10, 12, 15}
{0, 2, 3, 4, 6, 7, 9, 10, 12, 15}
{1, 2, 3, 4, 6, 7, 9, 10, 12, 15}
{3, 4, 6, 7, 9, 10, 12, 15}
{2, 4, 6, 7, 9, 10, 12, 15}
{2, 3, 6, 7, 9, 10, 12, 15}
{2, 3, 4, 5, 6, 7, 9, 10, 12, 15}
{2, 3, 4, 7, 9, 10, 12, 15}
{2, 3, 4, 6, 9, 10, 12, 15}
{2, 3, 4, 6, 7, 8, 9, 10, 12, 15}
{2

In [ ]:
### Lagrangian Relaxation ###

lagrange_multipliers = np.array([-34.7078948523, -69.7745408216, -35.0906265859, -33.7149585875, -47.8876574082, -50.6285183086, -28.5846408578, -34.9330479128, -34.1894563119, -28.6462026929, -44.3243916588, -27.0003565218, -22.2637721293, -43.3876767701, -34.9362503744, -27.8298171032, -29.1621882132, -35.6408950225, -33.7153478247, -53.8379875196, -51.8332308972, -50.0779878153, -20.8181085093, -17.8645208278, -18.8935645115, -56.3144024134, -26.8044926697, -28.3182144201, -42.7250751024, -69.7218769381, -36.5262904613, -38.0481861949])

def constructVarsLagrange():
    global noFacs , noCusts , dems , myLagrangeModel , yVarsLagr , xVarsLagr
    for i in range( noFacs ):
        curVar = myLagrangeModel.addVar( vtype = GRB.BINARY , ub = 1 , name = "y" + str( i ) )
        yVarsLagr[ i ] = curVar
    for i in range( noFacs ):
        for j in range( noCusts ):
            curVar = myLagrangeModel.addVar( vtype = GRB.CONTINUOUS , ub = dems[ j ] , name = "x" + str( i ) + "_" + str( j ) )
            xVarsLagr[ i ][ j ] = curVar
    myLagrangeModel.update()
    
def constructObjLagrange():
    global noFacs, noCusts, fixs , dems, myLagrangeModel, yVarsLagr, xVarsLagr, lagrange_multipliers
    objExpr = LinExpr()
    for i in range( noFacs ):
        objExpr += fixs[ i ] * yVarsLagr[ i ]
    for i in range( noFacs ):
        for j in range( noCusts ):
            objExpr += (distance( i , j ) + lagrange_multipliers[j]) * xVarsLagr[ i ][ j ]
    for j in range( noCusts ):
        objExpr -= lagrange_multipliers[j] * dems[ j ]
    myLagrangeModel.setObjective( objExpr , GRB.MINIMIZE )
    
def constructConstrsLagrange():
    global noFacs, noCusts, facs, caps , dems, myLagrangeModel, yVarsLagr, xVarsLagr
#     for j in range( noCusts ):
#         constrExpr = LinExpr()
#         for i in range( noFacs ):
#             constrExpr += 1 * xVarsLagr[ i ][ j ]
#         myLagrangeModel.addConstr( lhs = constrExpr , sense = GRB.EQUAL , rhs = dems[ j ] , name = "d" + str( j ) )
    for i in range( noFacs ):
        constrExpr = LinExpr()
        for j in range( noCusts ):
            constrExpr += 1 * xVarsLagr[ i ][ j ]
        constrExpr += ( - caps[ i ] * yVarsLagr[ i ] )
        myLagrangeModel.addConstr( lhs = constrExpr, sense = GRB.LESS_EQUAL , rhs = 0 , name= "c" + str( i ) )
    myLagrangeModel.update()
    
myLagrangeModel = Model( "lagrange_relaxation" )
yVarsLagr = [ 0 for i  in range( noFacs ) ]
xVarsLagr = [ [ 0 for j in range ( noCusts ) ] for i in range ( noFacs ) ]
constructVarsLagrange()
constructObjLagrange()
constructConstrsLagrange()
myLagrangeModel.write( "lagrange_relaxation.lp" )
myLagrangeModel.optimize()
print ( "Optimal objective: " + str( myLagrangeModel.ObjVal ) )

In [ ]:
### Integer Programming LP Relaxation ###

def constructVarsLPRelax():
    global noFacs , noCusts , dems , myLPModel , yVarsLP , xVarsLP
    for i in range( noFacs ):
        curVar = myLPModel.addVar( vtype = GRB.CONTINUOUS , ub = 1 , name = "y" + str( i ) )
        yVarsLP[ i ] = curVar
    for i in range( noFacs ):
        for j in range( noCusts ):
            curVar = myLPModel.addVar( vtype = GRB.CONTINUOUS , ub = dems[ j ] , name = "x" + str( i ) + "_" + str( j ) )
            xVarsLP[ i ][ j ] = curVar
    myLPModel.update()
    
def constructObjLPRelax():
    global noFacs, noCusts, fixs , myLPModel , yVarsLP , xVarsLP
    objExpr = LinExpr()
    for i in range( noFacs ):
        objExpr += fixs[ i ] * yVarsLP[ i ]
    for i in range( noFacs ):
        for j in range( noCusts ):
            objExpr += distance( i , j ) * xVarsLP[ i ][ j ]
    myLPModel.setObjective( objExpr , GRB.MINIMIZE )
    
def constructConstrsLPRelax():
    global noFacs, noCusts, facs, caps , dems, myLPModel , yVarsLP , xVarsLP
    for j in range( noCusts ):
        constrExpr = LinExpr()
        for i in range( noFacs ):
            constrExpr += 1 * xVarsLP[ i ][ j ]
        myLPModel.addConstr( lhs = constrExpr , sense = GRB.EQUAL , rhs = dems[ j ] , name = "d" + str( j ) )
    for i in range( noFacs ):
        constrExpr = LinExpr()
        for j in range( noCusts ):
            constrExpr += 1 * xVarsLP[ i ][ j ]
        constrExpr += ( - caps[ i ] * yVarsLP[ i ] )
        myLPModel.addConstr( lhs = constrExpr, sense = GRB.LESS_EQUAL , rhs = 0 , name= "c" + str( i ) )
    myLPModel.update()
    
myLPModel = Model( "LP_relaxation" )
yVarsLP = [ 0 for i  in range( noFacs ) ]
xVarsLP = [ [ 0 for j in range ( noCusts ) ] for i in range ( noFacs ) ]
constructVarsLPRelax()
constructObjLPRelax()
constructConstrsLPRelax()
myLPModel.write( "LP_relaxation.lp" )
myLPModel.optimize()
print ( "Optimal objective: " + str( myLPModel.ObjVal ) )